In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
import matplotlib.pyplot as plt
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from keras.layers import Conv2D, Input, LeakyReLU, Dense, Activation, Flatten, Dropout, MaxPool2D
from keras import models
from keras.optimizers import Adam,RMSprop 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [0]:
link_train = '/content/drive/My Drive/emnist-balanced-train.csv'
link_test = '/content/drive/My Drive/emnist-balanced-test.csv'

In [0]:
data_train = pd.read_csv(link_train, header=None)

data_test = pd.read_csv(link_test, header=None)

In [0]:
data_train.head(10)

In [0]:
class_mapping = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabdefghnqrt'

In [0]:
data_train.shape

In [0]:
data_train = data_train / 255.0
data_test = data_test / 255.0

In [0]:
plt.imshow(data_train.values[2, 1:].reshape([28, 28]), cmap='Greys_r')
plt.show()

img_flip = np.transpose(data_train.values[2,1:].reshape(28, 28), axes=[1,0])
plt.imshow(img_flip, cmap='Greys_r')
plt.show()

In [0]:
def show_img(data, row_num):
    img_flip = np.transpose(data.values[row_num,1:].reshape(28, 28), axes=[1,0])
    tit = int(data.values[row_num][0]*255)
    plt.title(str(tit), y=-0.15,color="green")
    plt.imshow(img_flip, cmap='Greys_r')
    print(class_mapping[tit])
  

In [0]:
show_img(data_train, 143)

In [0]:
num_classes = 47 
img_size = 28

def img_label_load(data_path, num_classes=None):
    data = pd.read_csv(data_path, header=None)
    data_rows = len(data)
    if not num_classes:
        num_classes = len(data[0].unique())
    
    # this assumes square imgs. Should be 28x28
    img_size = int(np.sqrt(len(data.iloc[0][1:])))
    
    # Images need to be transposed. This line also does the reshaping needed.
    imgs = np.transpose(data.values[:,1:].reshape(data_rows, img_size, img_size, 1), axes=[0,2,1,3]) # img_size * img_size arrays
    
    labels = keras.utils.to_categorical(data.values[:,0], num_classes) # one-hot encoding vectors
    
    return imgs/255., labels

In [0]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu',padding="same", input_shape=(img_size,img_size,1)))
model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu',padding="same"))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Dropout(.25))

model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding="same"))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding="same"))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Dropout(.25))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(units=num_classes, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

In [0]:
X, y = img_label_load(link_train)
print(X.shape)

In [0]:
data_generator = keras.preprocessing.image.ImageDataGenerator(validation_split=.2)

data_generator_with_aug = keras.preprocessing.image.ImageDataGenerator(featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images)
        

training_data_generator = data_generator.flow(X, y, subset='training')
validation_data_generator = data_generator.flow(X, y, subset='validation')
history = model.fit(training_data_generator, 
                              batch_size = 256, epochs=20,
                              validation_data=validation_data_generator)

In [0]:
test_X, test_y = img_label_load(link_test)
test_data_generator = data_generator.flow(X, y)

model.evaluate(test_data_generator)

3525/3525 [==============================] - 13s 4ms/step - loss: 0.1906 - accuracy: 0.9289


[0.19061018526554108, 0.9289095997810364]

In [0]:
show_img(data_test, 123)

In [0]:
X_test, y_test = img_label_load(link_test)

In [0]:
def run_prediction(idx):
    result = np.argmax(model.predict(X_test[idx:idx+1]))
    print('Prediction: ', result, ', Char: ', class_mapping[result])
    print('Label: ', data_test.values[idx,0]*255)
    show_img(data_test, idx)

In [0]:
import random

for _ in range(1,10):
    idx = random.randint(0, 47-1)
    run_prediction(idx)

In [0]:
show_img(data_test, 144)
np.argmax(y_test[144])

In [0]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

f = open('model.tflite', "wb")
f.write(tflite_model)
f.close()

In [0]:
try:
  from google.colab import files
  files.download('model.tflite')
except ImportError:
  import os
  print('TF Lite model:', os.path.join(os.getcwd(), 'model.tflite'))